Final Submit DAW
<br>
Authors: Mychailo & Roberto

#  **New York State of Mind**

##  **Introduction**

This notebook presents the **final report** for our project in the module  
**Data Wrangling (DAW)**.  

It is based on two primary datasets:  

- [NYC 311 Service Requests](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/about_data)  
- [StreetEasy Data Dashboard](https://streeteasy.com/blog/data-dashboard/?utm_source=chatgpt.com)  

---

The project demonstrates a complete **data wrangling pipeline**, covering the topics of the LE's in this module.

1. **Importing** – retrieving and sampling real-world open data  
2. **Data Cleaning** – preparing data for analysis  
3. **Tranforming** – further preparation feature engineering  
4. **Joining** - joining the data
5. **Data Pipelines** – bla bla
6. **Reproducability** - bla bla

---

Our work follows the structure of the **commonly used data exploration framework**, 

---
<img src="./img/data_exploring.png">


## LE1 Importing
Whole ass explenation on how to import

In [ ]:
# Importing Data
# TODO: Actually call the function and import

### Testing the import of the data

In [ ]:
# Testing the import 
# TODO: Test the import 